# MCMC2.0: Iterators, Generators, and Channels

Iterators and generators are two important tools in the modern functional programming and Julia implements new "channels." I will show a typical implementation of them for MCMC.

## Iterators

Iterators are often used to save memories. If we wish to do something to each element of a large list, we do not have to store all the elements between the processes. Iterators return each element one by one, so we can do opearations one by one and each time we can discard old data to save memories. Actually, we already used iterators in usual `for` loops.

In [1]:
iter = 1 : 2

1:2

This is already iterator counting from 1 to 2.

In [2]:
element, state = iterate(iter)

(1, 1)

This is the initial state of the iterator `1:2`.

In [3]:
element, state = iterate(iter, state)

(2, 2)

These are the next item and the next state.

In [4]:
result = iterate(iter, state)
result == nothing

true

If there is nothing to iterate, they return nothing.

In [5]:
iter = 1 : 5
for i in iter
    println(i)
end

1
2
3
4
5


This is the same as follow:

In [6]:
iter_result = iterate(iter)
while iter_result !== nothing
    (i, state) = iter_result
    println(i)
    iter_result = iterate(iter, state)
end

1
2
3
4
5


If we use an iterator instead of a list, we do not need to store the whole list anymore.

In [7]:
list = collect(iter)
for i in 1 : length(list)
    println(list[i] ^ 2)
end

1
4
9
16
25


This is meaningless and should be rewritten as:

In [8]:
for i in iter
    println(i ^ 2)
end

1
4
9
16
25


In this way, we can (trivially) avoid allocating a memory for the list. Or this works if the memory does not matter:

In [9]:
for i in list
    println(i ^ 2)
end

1
4
9
16
25


## IterTools

~ under construction ~

In [11]:
using IterTools
using Distributions
"""
Curried form of a Gibbs sampler
"""
function Gibbs1(a::Float64, b::Float64, c::Float64)::Function
    xy::Tuple{Float64, Float64} -> begin
        x = rand(Normal(b / a * xy[2], 1 / sqrt(a)))
        x, rand(Normal(b / c * x, 1 / sqrt(c)))
    end
end
samples = iterated(Gibbs1(1.0, 0.8, 1.0), (0.0, 0.0))
collect(Iterators.take(samples, 10))

10-element Array{Tuple{Float64,Float64},1}:
 (0.0, 0.0)
 (0.9127951497017784, -0.6284347179702566)
 (-1.5749606556547757, -2.0547765394672077)
 (-0.9151622894614292, -0.9684150802224576)
 (-2.113997770135818, -1.8678843794523343)
 (-0.09565712578099994, 0.18278799663206857)
 (1.3173686575459462, 0.6528062457357966)
 (1.0720724377152722, -0.48279966693868837)
 (-1.274205707459569, -2.20457344333387)
 (-2.0015486315965445, -2.467314814119539)

## Generator 1 (generator expression)

An iterator is most useful when combined with a generator. We generally call something creating an iterator a generator. Generators in general include a generator expression, a generator function, etc. Julia supports a generator expression as follows:

In [12]:
gene = (x^2 for x in 1 : 5)

Base.Generator{UnitRange{Int64},var"#5#6"}(var"#5#6"(), 1:5)

This is very similar to a list comprehension:

In [13]:
list = [x^2 for x in 1 : 5]

5-element Array{Int64,1}:
  1
  4
  9
 16
 25

The list comprehension will allocate a memory for the whole list. The generator will save your memory.

In [14]:
for i in gene
    println(i)
end

1
4
9
16
25


## Generator 2 (generator function = resumable function)

An easier, simpler, and most useful way to create an iterator/generator is a generator function, and Python and most modern language support this type of functions. Julia does not natively support Python-type generator functions, but simply you can call ResumableFunctions.jl. This is the easiest way to reproduce the behavior of generator functions.

In [15]:
using ResumableFunctions
@resumable function Ising()::Vector{Int64}
    N = 8
    σ = ones(Int64, N)
    β = 1.0
    while true
        for i in 1 : 1000
            j = rand(1 : N)
            ΔβE = 2β * σ[j] * (σ[mod1(j + 1, N)] + σ[mod1(j - 1, N)])
            -ΔβE > log(rand()) && (σ[j] = -σ[j])
        end
        @yield σ
    end
end

Ising (generic function with 1 method)

With `@resumable` macro, you can define a resumable function. This function does not have `return`, but yield what you want at `@yield` points. This function directely returns an iterater, i.e. a function iterate(iter) is already defined when we call `iter = Ising()`. Then, this iterater gives you the "yield" repeatedly each time you reached `@yield` point.

In [16]:
iter = Ising()
@show iter();

iter() = [-1, -1, -1, -1, 1, 1, -1, -1]


In [17]:
for i in 1 : 10
    println(iter())
end

[-1, -1, -1, -1, -1, -1, -1, -1]
[1, 1, 1, 1, 1, 1, 1, 1]
[-1, -1, -1, -1, -1, -1, -1, -1]
[-1, -1, -1, 1, 1, 1, 1, -1]
[-1, -1, -1, -1, 1, 1, -1, -1]
[-1, -1, -1, -1, -1, -1, -1, -1]
[1, 1, 1, 1, 1, 1, 1, 1]
[-1, -1, 1, 1, 1, 1, -1, -1]
[1, 1, 1, 1, 1, -1, -1, 1]
[1, 1, 1, 1, 1, 1, 1, 1]


ResummableFunctions are very fast and memory-efficient because the macro `@resummable` will rewrite a function into a normal iterator directly before the JIT compilation.

## Channels

Instead of a generator function, Julia natively supports Channels for parallel computing. Especially, `Channel` is a stronger and more general tool to create an iterator from the function. This is a generalized concept of a generator function, but in most cases ResumableFunctions.jl is faster and memory-efficient. However, if you need a postprocess based on lazy evaluation, Channels work better from my experience.

In [18]:
"""
Channel form of a Gibbs sampler
"""
function Gibbs2(a::Float64, b::Float64, c::Float64)::Channel
    Channel(ctype = Tuple{Float64,Float64}) do channel::Channel{Tuple{Float64,Float64}}
        N = 10
        x = 0.0
        y = 0.0
        put!(channel, (x, y))
        for i in 1 : N
            x = rand(Normal(b / a * y, 1 / sqrt(a)))
            y = rand(Normal(b / c * x, 1 / sqrt(c)))
            put!(channel, (x, y))
        end
    end
end

Gibbs2

In [19]:
for z in Gibbs2(1.0, 0.8, 1.0)
    println(z)
end

(0.0, 0.0)
(-1.4483067527162923, -3.288860070333487)
(-1.5990998964758119, -0.27948548940012274)
(-0.617979911385413, 1.2058631082609041)
(0.12432459915672833, -1.2702030925724037)
(-0.5603761412417363, -2.207730393547431)
(-0.3468902755565808, 0.5542581410676211)
(1.1266390318305852, 1.6294321172774313)
(1.7758541439554818, -0.27200753027353897)
(0.5718912133307947, 0.8062453380022676)
(1.230502386037514, -1.2842891054252608)


This code is acceptable if the number of tasks is just one.

~ under construction ~

In [20]:
"""
Channel form of the first part of a Gibbs sampler
"""
function xupdate(a::Float64, b::Float64, c::Float64, input::Channel{Tuple{Float64,Float64}}, output::Channel{Tuple{Float64,Float64}})
    N = 10
    x = 0.0
    y = 0.0
    for i in 1 : N
        (x, y) = take!(input)
        x = rand(Normal(b / a * y, 1 / sqrt(a)))
        put!(output, (x, y))
    end
    close(output)
end
"""
Channel form of the second part of a Gibbs sampler
"""
function yupdate(a::Float64, b::Float64, c::Float64, input::Channel{Tuple{Float64,Float64}}, output::Channel{Tuple{Float64,Float64}})
    N = 10
    x = 0.0
    y = 0.0
    for i in 1 : N
        (x, y) = take!(input)
        y = rand(Normal(b / c * x, 1 / sqrt(c)))
        println((x, y))
        put!(output, (x, y))
    end
    close(output)
end
ch1 = Channel{Tuple{Float64,Float64}}(32)
ch2 = Channel{Tuple{Float64,Float64}}(32)
@async xupdate(1.0, 0.8, 1.0, ch1, ch2)
@async yupdate(1.0, 0.8, 1.0, ch2, ch1)
put!(ch1, (0.0, 0.0));

(-0.15361505949416265, -0.10029227247033887)
(-0.1693931653401406, 1.8232576698744671)
(1.2540636340969513, 2.0894050963200828)
(2.566168444797236, 0.20919170907748152)
(-0.18842645515874068, -1.313330404770729)
(-1.2049902117741285, -0.11491832616619713)
(-0.3070781618191319, -1.130872264929062)
(-1.9299073685690173, -2.7557361958603996)
(-4.017750962464292, -0.28939871055812816)
(0.3773937354793355, -0.8795587582992834)


The difference bewteen Channels and ResumableFunctions lies in how they compile a code. Check: https://white.ucc.asn.au/2017/11/18/Lazy-Sequences-in-Julia.html

## RemoteChannels

It will be discussed in MCMC4.0.

**Exercise**: implement takewhile and dropwhile for iterators. Look at Laze.jl for more details.These functions are sometimes useful for MCMC.

Note: Iterators.takewhile and Iterators.dropwhile are supported from Julia 1.4.0.